# Calculadora detallada de sou net

En aquest document podràs calcular i visualitzar clarament quins impostos pagues, quin sou et queda mensual i quan t'incrementa el sou mensual quan et pugen el sou.

**Instruccions d'us:** Executa totes les cel·les en ordre (o premer "Run all"), a l'arribar la cel·la final, omple els camps i calcula el teu sou net.


In [11]:
#@title Imports & Utils

from dataclasses import dataclass, field
from decimal import Decimal, ROUND_HALF_UP, getcontext
from typing import List, Tuple, Optional, Dict

# Increse precission
getcontext().prec = 28

# Forma d'arrodoniment per defecte: arrodonir a 2 decimals, meitat endavant (banker's? no: half-up)
def round_euro(x: Decimal) -> Decimal:
    return x.quantize(Decimal("0.01"), rounding=ROUND_HALF_UP)


In [12]:
#@title Seguretat Social (valors públics 2025)
#@markdown Editar la cel·la per modificar: Bases màximes i mínimes de cotització per grup i tipus de cotització

# -------------------------
# Seguretat Social 2025
# -------------------------

# Bases màximes
SS_BASE_MAX_MONTHLY = Decimal("4909.50")
SS_BASE_MAX_DAILY = Decimal("163.65")

# Bases mínimes per grup
SS_BASE_MIN_BY_GROUP = {
    "Sou mensual; adult; Enginyeres i llicenciades universitàries": Decimal("1929.00"),
    "Sou mensual; adult; Enginyers tècnics, perits i ajudants titulats": Decimal("1599.60"),
    "Sou mensual; adult; Caps administratius i de taller": Decimal("1391.70"),
    "Sou mensual; adult; altres": Decimal("1381.20"),
    "Base diaria; adult": Decimal("46.04"),
    "Menor d'edat": Decimal("46.04"),
}

# Tipus de cotització (treballador)
DEFAULT_SS_RATES = {
    "contingencies_common_worker": Decimal("0.0470"),
    "unemployment_worker_indefinite": Decimal("0.0155"),
    "unemployment_worker_temporary": Decimal("0.0160"),
    "training_worker": Decimal("0.0010"),
    "mei_worker": Decimal("0.0013"),
}


In [13]:
#@title Mínim personal / familiar (valors públics 2025)
#@markdown Editar la cel·la per modificar: Mínim contribuent general, i els seus increments per edat, fills o discapacitats

MINIMO_CONTRIBUYENTE = Decimal("5550.00")  # contribuent general

# Increment per edat
AGE_ADJUSTMENTS = [
    (65, Decimal("1150")),  # +1150 € si >65
    (75, Decimal("2550")),  # +2550 € si >75
]

CHILDREN_UNDER_25_ADJUSTMENT = [
    (1, Decimal("2400")),  # primer fill
    (2, Decimal("2700")),  # segon fill
    (3, Decimal("4000")),  # tercer i següents
]
ASCENDENTS_ADJUSTMENT = [Decimal("1150"), Decimal("1400"), Decimal("1500")]


DISABILITY_SELF_ADJUSTMENTS = [
    (33, 3000),
    (65, 9000),
]
DISABILITY_RELATIVES_ADJUSTMENTS = [
    (33, 1000),
    (65, 2000),
]



In [21]:
#@title Escala IRPF estatal i autonòmica (valors públics 2024)
#@markdown Editar la cel·la per modificar les escales d'IRPF estatal i autonòmiques

IRPF_SCALE_ESTATAL = [
            (0, 12450, Decimal("0.095")),
            (12450, 20200, Decimal("0.12")),
            (20200, 35200, Decimal("0.15")),
            (35200, 60000, Decimal("0.185")),
            (60000, 300000, Decimal("0.225")),
            (300000, None, Decimal("0.245")),

### Comunitats autònomes

IRPF_SCALE_CATALUNYA = [
            (0, 12450, Decimal("0.12")),
            (12450, 17707.20, Decimal("0.12")),
            (17707.20, 21000, Decimal("0.14")),
            (21000, 33007.20, Decimal("0.15")),
            (33007.20, 53407.20, Decimal("0.188")),
            (53407.20, 90000, Decimal("0.215")),
            (90000, None, Decimal("0.235")),
        ]


###########################
# TODO: Augmentar CCOO
###########################


SyntaxError: '[' was never closed (ipython-input-922707709.py, line 4)

In [31]:
#@title Estructures de dades

@dataclass
class FamilySituation:
    age: int = 30
    children_ages: List[int] = field(default_factory=list)
    children_disabilities: List[int] = field(default_factory=list)
    ascendents_dependent: int = 0
    disability_percent_self: int = 0
    disability_percent_relatives: int = 0

    def minimo_personal_familiar(self) -> Decimal:
        """
        Calcula el mínim personal i familiar segons AEAT 2025.
        Tot el que pot canviar està separat en constants globals.
        """
        minimo = MINIMO_CONTRIBUYENTE

        # 1) Ajust per edat del contribuent
        for age_limit, inc in AGE_ADJUSTMENTS:
            if self.age > age_limit:
                minimo += inc

        # 2) Ajust per fills
        for i, child_age in enumerate(self.children_ages):
            if child_age <= 25:
                child_order = i + 1
                increment = None
                for order_limit, value in CHILDREN_UNDER_25_ADJUSTMENT:
                    if child_order <= order_limit:
                        increment = value
                        break
                if increment is None:
                    increment = CHILDREN_UNDER_25_ADJUSTMENT[-1][1]
                minimo += increment

        # 3) Ascendents dependents
        for i in range(self.ascendents_dependent):
            if i < len(ASCENDENTS_ADJUSTMENT):
                minimo += ASCENDENTS_ADJUSTMENT[i]
            else:
                minimo += ASCENDENTS_ADJUSTMENT[-1]

        # 4) Discapacitat del contribuent
        for perc_limit, inc in DISABILITY_SELF_ADJUSTMENTS:
            if self.disability_percent_self >= perc_limit:
                minimo += inc

        # 5) Discapacitat de familiars dependents
        for perc_limit, inc in DISABILITY_RELATIVES_ADJUSTMENTS:
            if self.disability_percent_relatives >= perc_limit:
                minimo += inc

        return minimo




@dataclass
class IRPFScale:
    """
    Representa l'escala d'IRPF com a llista de trams:
    List[ (from_exclusive_or_inclusive, to_inclusive_or_None, rate_decimal) ]
    """
    brackets: List[Tuple[Decimal, Optional[Decimal], Decimal]] = field(default_factory=list)

    def tax_on_base(self, base: Decimal) -> Decimal:
        """
        Aplica l'escala progressiva i retorna l'import total d'impost (sense deduccions).
        """
        tax = Decimal("0")
        for low, high, rate in self.brackets:
            low = Decimal(low)
            high = Decimal(high) if high is not None else None
            if base <= low:
                break
            upper = base if (high is None or base < high) else high
            taxable = upper - low
            if taxable <= 0:
                continue
            tax += taxable * rate
        return tax

    @classmethod
    def combined_scale(cls, regional_scale: List[Tuple[Decimal, Optional[Decimal], Decimal]],
                       state_scale: List[Tuple[Decimal, Optional[Decimal], Decimal]]) -> "IRPFScale":
        # Collect all breakpoints
        breakpoints = set()
        for low, high, _ in regional_scale + state_scale:
            breakpoints.add(low)
            if high is not None:
                breakpoints.add(high)
        breakpoints = sorted(breakpoints)

        combined_brackets = []
        for i in range(len(breakpoints) - 1):
            low = Decimal(breakpoints[i])
            high = Decimal(breakpoints[i+1])

            # Find rates for this interval
            r_rate = next((rate for l, h, rate in regional_scale if l <= low < (h if h is not None else Decimal("1e18"))), Decimal(0))
            s_rate = next((rate for l, h, rate in state_scale if l <= low < (h if h is not None else Decimal("1e18"))), Decimal(0))

            combined_brackets.append((low, high, r_rate + s_rate))

        # Handle last infinite bracket if any
        last_r_rate = next((rate for l, h, rate in regional_scale if l <= breakpoints[-1] < (h if h is not None else Decimal("1e18"))), Decimal(0))
        last_s_rate = next((rate for l, h, rate in state_scale if l <= breakpoints[-1] < (h if h is not None else Decimal("1e18"))), Decimal(0))
        combined_brackets.append((Decimal(breakpoints[-1]), None, last_r_rate + last_s_rate))

        return cls(combined_brackets)


# -------------------------
# Funcions de càlcul de SS
# -------------------------
def apply_base_limits(base: Decimal,
                      base_min: Decimal,
                      base_max: Decimal,
                      is_daily: bool = False,
                      days_in_month: int = 30) -> Decimal:
    """
    Ajusta la base entre mínim i màxim legals.
    Si és base diària, aplica mínim/màxim diari i després multiplica pels dies del mes.
    """
    if is_daily:
        base_diari = base / Decimal(days_in_month)
        base_diari_ajustat = max(base_min, min(base_diari, base_max))
        return base_diari_ajustat * Decimal(days_in_month)
    else:
        return max(base_min, min(base, base_max))


# def calculate_social_security_worker(monthly_base: Decimal,
#                                       ss_rates: Dict[str, Decimal] = DEFAULT_SS_RATES,
#                                       contract_type: str = "indefinite",
#                                       base_min: Decimal = SS_BASE_MIN_GENERIC_2025,
#                                       base_max: Decimal = SS_BASE_MAX_2025,
#                                       is_daily: bool = False,
#                                       days_in_month: int = 30,
#                                       rounding: bool = True) -> Decimal:
#     # Ajust base segons mensual/diari
#     base = apply_base_limits(monthly_base, base_min, base_max, is_daily=is_daily, days_in_month=days_in_month)

#     # Tipus d’atur segons contracte
#     t_des = ss_rates["unemployment_worker_indefinite"] if contract_type == "indefinite" else ss_rates["unemployment_worker_temporary"]

#     total_rate = ss_rates["contingencies_common_worker"] + t_des + ss_rates["training_worker"] + ss_rates["mei_worker"]
#     cotitzacio = base * total_rate
#     if rounding:
#         cotitzacio = round_euro(cotitzacio)
#     return cotitzacio


# -------------------------
# Funcions IRPF (base anual i retenció)
# -------------------------
def calculate_base_imposable_irpf(annual_gross: Decimal,
                                  annual_employee_ss: Decimal,
                                  family: FamilySituation,
                                  other_deductions: Decimal = Decimal("0")) -> Decimal:
    """
    Base imposable anual simplificada per l'IRPF:
    base_imposable = brut_anual - cotitzacions_anuals - minim_personal_i_familiar - altres deduccions
    Observació: l'AEAT aplica criteris addicionals (liquidable, etc.); funció simplificada.
    """
    minimo_pf = family.minimo_personal_familiar()
    base = annual_gross - annual_employee_ss - minimo_pf - other_deductions
    if base < 0:
        base = Decimal("0")
    return base




In [32]:
#@title #### Definició pie plot
import matplotlib.pyplot as plt
from decimal import Decimal
from typing import Dict

def plot_net_pay_and_taxes(
    gross_including_benefits: Decimal,
    net_per_paga: Decimal,
    n_pagues: int,
    cotitzacions_anuals: Decimal,
    irpf_anual: Decimal,
    ss_contingencies_comunes_annual: Decimal,
    t_des_annual: Decimal,
    ss_training_annual: Decimal,
    ss_mei_annual: Decimal
):
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))

    # Chart 1: Net Pay vs Taxes
    total_taxes_annual = cotitzacions_anuals + irpf_anual
    net_annual = net_per_paga * Decimal(n_pagues)
    net_vs_taxes_labels = ["Sou Net Anual", "Impostos i Cotitzacions Anuals"]
    net_vs_taxes_values = [float(net_annual), float(total_taxes_annual)]
    colors1 = ["#99ff99", "#ff6666"]

    axes[0].pie(net_vs_taxes_values, autopct='%1.1f%%', startangle=90, colors=colors1)
    axes[0].set_title("Distribució Sou Net vs Impostos Anuals")
    axes[0].legend(net_vs_taxes_labels, loc="best")

    # Chart 2: Tax Breakdown
    taxes_breakdown_labels = [
        "IRPF",
        "SS: Contingències Comunes",
        "SS: Atur",
        "SS: Formació Professional",
        "SS: MEI"
    ]
    taxes_breakdown_values = [
        float(irpf_anual),
        float(ss_contingencies_comunes_annual),
        float(t_des_annual),
        float(ss_training_annual),
        float(ss_mei_annual)
    ]
    colors2 = ["#ff9999", "#66b3ff", "#6666ff", "#ffcc99", "#c2c2f0"]

    axes[1].pie(taxes_breakdown_values, autopct='%1.1f%%', startangle=90, colors=colors2)
    axes[1].set_title("Distribució dels impostos i cotitzacions anuals")
    axes[1].legend(taxes_breakdown_labels, loc="best")

    plt.tight_layout()
    plt.show()


In [33]:
#@title bar plot
import matplotlib.pyplot as plt
import numpy as np

def plot_salary_blocks(
    gross: Decimal,
    n_pagues: int,
    pagues_prorratejades: bool,
    retribucio_en_especie_ann: Decimal,
    grup_cotitzacio: str,
    contract_type: str,
    fam: FamilySituation,
    region: str,
    other_deductions: Decimal
):
    first_block_gross = Decimal("10000")
    remaining_gross = gross - first_block_gross

    n_remaining_blocks = int(remaining_gross // 1000) + 1 if remaining_gross > 0 else 0
    gross_increments = [first_block_gross] + [first_block_gross + Decimal(1000)*(i+1) for i in range(n_remaining_blocks)]
    gross_increments[-1] = gross  # assegurar l'últim bloc exactament

    net_blocks = []
    taxes_blocks_breakdown = []

    for i, current_gross in enumerate(gross_increments):
        previous_gross = gross_increments[i-1] if i > 0 else Decimal(0)

        current_result = compute_net_pay(
            basic_annual_gross=current_gross,
            n_pagues=n_pagues,
            pagues_prorratejades=pagues_prorratejades,
            retribucio_en_especie_ann=retribucio_en_especie_ann,
            grup_cotització=grup_cotitzacio,
            contract_type=contract_type,
            family=fam,
            region=region,
            other_annual_deductions=other_deductions,
            show_output=False
        )

        prev_result = compute_net_pay(
            basic_annual_gross=previous_gross,
            n_pagues=n_pagues,
            pagues_prorratejades=pagues_prorratejades,
            retribucio_en_especie_ann=retribucio_en_especie_ann,
            grup_cotització=grup_cotitzacio,
            contract_type=contract_type,
            family=fam,
            region=region,
            other_annual_deductions=other_deductions,
            show_output=False
        ) if i > 0 else {
            "sou_net_per_paga": Decimal(0),
            "irpf_anual": Decimal(0),
            "ss_contingencies_comunes_annual": Decimal(0),
            "t_des_annual": Decimal(0),
            "ss_training_annual": Decimal(0),
            "ss_mei_annual": Decimal(0)
        }

        current_net_annual = current_result["sou_net_per_paga"] * Decimal(n_pagues)
        prev_net_annual = prev_result["sou_net_per_paga"] * Decimal(n_pagues)

        if i == 0:
            net_blocks.append(float((current_net_annual - prev_net_annual)/10))
        else:
            net_blocks.append(float(current_net_annual - prev_net_annual))

        taxes_increment = [
            float(current_result["irpf_anual"] - prev_result["irpf_anual"]),
            float(current_result["ss_contingencies_comunes_annual"] - prev_result["ss_contingencies_comunes_annual"]),
            float(current_result["t_des_annual"] - prev_result["t_des_annual"]),
            float(current_result["ss_training_annual"] - prev_result["ss_training_annual"]),
            float(current_result["ss_mei_annual"] - prev_result["ss_mei_annual"])
        ]
        if i == 0:
            taxes_increment = [x/10 for x in taxes_increment]
        taxes_blocks_breakdown.append(taxes_increment)

    # ---------------------------
    # Crear posicions de les barres
    # ---------------------------
    bar_widths = [0.8*10] + [0.8]*(len(net_blocks)-1)
    bar_positions = [0]
    gap = 0.3
    for i in range(1, len(net_blocks)):
        prev_pos = bar_positions[i-1]
        bar_positions.append(prev_pos + (bar_widths[i-1] + bar_widths[i])/2 + gap)

    tax_colors = ["#ff9999", "#66b3ff", "#6666ff", "#ffcc99", "#c2c2f0"]

    # ---------------------------
    # Plot
    # ---------------------------
    fig, ax1 = plt.subplots(figsize=(14,6))

    # Stacked bars
    for i in range(len(net_blocks)):
        bottom = 0
        # Net part
        ax1.bar(bar_positions[i], net_blocks[i], bar_widths[i], color="#99ff99", label="Sou net" if i==0 else "")
        bottom = net_blocks[i]
        # Taxes part
        for j, tax in enumerate(taxes_blocks_breakdown[i]):
            ax1.bar(bar_positions[i], tax, bar_widths[i], bottom=bottom, color=tax_colors[j],
                    label=["IRPF", "SS: Contingències Comunes", "SS: Atur", "SS: Formació", "SS: MEI"][j] if i==0 else "")
            bottom += tax

    ax1.set_xlabel("Blocs de sou brut anual")
    ax1.set_ylabel("Euros")
    ax1.set_title("Distribució del sou net i impostos (primer bloc 10k€ més ample)")

    # Eix secundari amb percentatges
    ax2 = ax1.twinx()
    max_val = max([sum(block) + net for block, net in zip(taxes_blocks_breakdown, net_blocks)])
    ax2.set_ylim(ax1.get_ylim())
    ax2.set_ylabel("Percentatge respecte sou brut")
    ax2.set_yticks(np.arange(0, 1.1, 0.1)*ax1.get_ylim()[1])
    ax2.set_yticklabels([f"{int(p*100)}%" for p in np.arange(0,1.1,0.1)])

    # Eix X amb inclinació
    ax1.set_xticks(bar_positions)
    ax1.set_xticklabels([f'{gross_increments[i]:,.0f}' for i in range(len(gross_increments))],
                        rotation=35, ha="right", fontsize=9)

    ax1.grid(axis="y", linestyle="--", alpha=0.5)
    ax1.legend(loc="upper left", bbox_to_anchor=(1.02,1))
    plt.tight_layout()
    plt.show()


In [34]:
#@title Funció principal
import matplotlib.pyplot as plt
from decimal import Decimal
from typing import Dict

def compute_net_pay(
    basic_annual_gross: Decimal,
    n_pagues: int = 12,
    pagues_prorratejades: bool = True,
    retribucio_en_especie_ann: Decimal = Decimal("0"),
    grup_cotització: str = "Sou mensual; adult; Enginyeres i llicenciades universitàries",
    contract_type: str = "indefinite",
    family: FamilySituation = FamilySituation(),
    region: str = "catalunya",
    ss_rates: Dict[str, Decimal] = DEFAULT_SS_RATES,
    other_annual_deductions: Decimal = Decimal("0"),
    rounding: bool = True,
    show_output: bool = True  # New parameter to control output
) -> Dict[str, Decimal]:

    ss_base_min = SS_BASE_MIN_BY_GROUP[grup_cotització]
    ss_base_max = SS_BASE_MAX_MONTHLY

    assert region.lower()=="catalunya", "De moment només la regió de Catalunya està disponible"

    # Escala IRPF combinada
    if region.lower() == "catalunya":
        irpf_scale = IRPFScale.combined_scale(IRPF_SCALE_CATALUNYA,IRPF_SCALE_ESTATAL )

    gross_including_benefits = basic_annual_gross + retribucio_en_especie_ann
    monthly_base = gross_including_benefits / Decimal(n_pagues if pagues_prorratejades else 12)

    is_daily = grup_cotització in ["Base diaria; adult", "Menor d'edat"]

    # Calculate individual SS components
    if monthly_base < ss_base_min:
        base_ss_adjusted = monthly_base  # no aplicar mínim legal si sou real molt baix
    else:
        base_ss_adjusted = apply_base_limits(
            monthly_base,
            ss_base_min,
            ss_base_max if not is_daily else SS_BASE_MAX_DAILY,
            is_daily=is_daily,
            days_in_month=30
        )

    ss_contingencies_comunes_monthly = base_ss_adjusted * ss_rates["contingencies_common_worker"]
    t_des_monthly = base_ss_adjusted * (ss_rates["unemployment_worker_indefinite"] if contract_type == "indefinite" else ss_rates["unemployment_worker_temporary"])
    ss_training_monthly = base_ss_adjusted * ss_rates["training_worker"]
    ss_mei_monthly = base_ss_adjusted * ss_rates["mei_worker"]

    cotitzacions_mensuals = ss_contingencies_comunes_monthly + t_des_monthly + ss_training_monthly + ss_mei_monthly

    if rounding:
        cotitzacions_mensuals = round_euro(cotitzacions_mensuals)
        ss_contingencies_comunes_monthly = round_euro(ss_contingencies_comunes_monthly)
        t_des_monthly = round_euro(t_des_monthly)
        ss_training_monthly = round_euro(ss_training_monthly)
        ss_mei_monthly = round_euro(ss_mei_monthly)


    cotitzacions_anuals = cotitzacions_mensuals * Decimal(n_pagues)
    ss_contingencies_comunes_annual = ss_contingencies_comunes_monthly * Decimal(n_pagues)
    t_des_annual = t_des_monthly * Decimal(n_pagues)
    ss_training_annual = ss_training_monthly * Decimal(n_pagues)
    ss_mei_annual = ss_mei_monthly * Decimal(n_pagues)


    base_imposable = calculate_base_imposable_irpf(
        gross_including_benefits, cotitzacions_anuals, family, other_deductions=other_annual_deductions
    )

    irpf_anual = irpf_scale.tax_on_base(base_imposable)
    if rounding:
        irpf_anual = round_euro(irpf_anual)

    irpf_per_paga = irpf_anual / Decimal(n_pagues)
    if rounding:
        irpf_per_paga = round_euro(irpf_per_paga)

    gross_per_paga = gross_including_benefits / Decimal(n_pagues)
    net_per_paga = gross_per_paga - cotitzacions_mensuals - irpf_per_paga
    if rounding:
        net_per_paga = round_euro(net_per_paga)

    net_monthly_equivalent = net_per_paga * Decimal(n_pagues) / Decimal(12) if n_pagues != 12 else net_per_paga

    if show_output:
        plot_net_pay_and_taxes(
            gross_including_benefits,
            net_per_paga,
            n_pagues,
            cotitzacions_anuals,
            irpf_anual,
            ss_contingencies_comunes_annual,
            t_des_annual,
            ss_training_annual,
            ss_mei_annual
        )

        plot_salary_blocks(
            gross_including_benefits,
            n_pagues,
            pagues_prorratejades,
            retribucio_en_especie_ann,
            grup_cotització,
            contract_type,
            family,
            region,
            other_annual_deductions
        )


        print("=== RESUM DEL SOU ANUAL I IMPOSTOS ===")
        print(f"SOU BRUT ANUAL TOTAL (incloent retribució en espècie): {gross_including_benefits:.2f} €")
        print(f"Base mensual per cotitzacions a la Seguretat Social: {monthly_base:.2f} €")
        print(f"Cotitzacions anuals a la Seguretat Social (empleat): {cotitzacions_anuals:.2f} €")
        print(f"  - Contingències Comunes: {ss_contingencies_comunes_annual:.2f} €")
        print(f"  - Atur: {t_des_annual:.2f} €")
        print(f"  - Formació Professional: {ss_training_annual:.2f} €")
        print(f"  - MEI: {ss_mei_annual:.2f} €")
        print(f"Base imposable per l'Impost sobre la Renda: {base_imposable:.2f} €")
        print(f"IRPF anual: {irpf_anual:.2f} €")
        print(f"SOU NET PER PAGA: {net_per_paga:.2f} €")
        print(f"SOU NET MENSUAL EQUIVALENT: {net_monthly_equivalent:.2f} €")
        print("=======================================")


    return {
        "sou_brut_anual_total": gross_including_benefits,
        "base_mensual_per_ss": monthly_base,
        "cotitzacions_anuals_empleat": cotitzacions_anuals,
        "ss_contingencies_comunes_annual": ss_contingencies_comunes_annual,
        "t_des_annual": t_des_annual,
        "ss_training_annual": ss_training_annual,
        "ss_mei_annual": ss_mei_annual,
        "base_imposable_irpf": base_imposable,
        "irpf_anual": irpf_anual,
        "sou_net_per_paga": net_per_paga,
        "sou_net_mensual_equivalent": net_monthly_equivalent,
    }

In [35]:
#@title Càlcul del sou
from re import M
from IPython.display import display, Markdown
import ipywidgets as widgets
from decimal import Decimal
import matplotlib.pyplot as plt
import numpy as np

# -------------------------
# Títol general
# -------------------------
display(Markdown("## 🔹 Calculadora de sou net 2025"))
display(Markdown("Omple les dades a continuació. Prem el botó 'Calcular sou net' quan hagis acabat."))

# Layout general per a tots els widgets
desc_width = '200px'
widget_width = '600px'

# -------------------------
# Comunitat autònoma
# -------------------------
comunitats = [
    "Andalusia","Aragó","Astúries","Balears","Canàries","Cantàbria",
    "Castella-La Manxa","Castella i Lleó","Catalunya","Extremadura",
    "Galícia","La Rioja","Madrid","Murcia","Navarra","País Basc","València"
]
display(Markdown("### 🌍 Comunitat autònoma"))
display(Markdown("Selecciona la teva comunitat per aplicar l'IRPF correcte (estat + autonòmic)."))
display(Markdown("ALERTA! De moment només funciona Catalunya."))
ca_selector = widgets.Dropdown(
    options=comunitats,
    value="Catalunya",
    description="Comunitat:",
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)
display(ca_selector)

# -------------------------
# Dades salarials
# -------------------------
display(Markdown("### 💰 Dades salarials"))

annual_gross_input = widgets.FloatText(
    value=25000,
    description='Sou brut anual (€):',
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)

n_pagues_input = widgets.IntSlider(
    value=14, min=12, max=15, step=1,
    description='Nombre de pagues:',
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)
display(Markdown("Si les pagues extraordinàries estan repartides, no ho marquis si cobres les pagues extraordinàries per seperat"))

pagues_prorratejades_input = widgets.Checkbox(
    value=False,
    description='Pagues prorratejades',
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)

display(Markdown("Valors que reps de l'empresa no en efectiu. p.e cotxe, bons"))

retribucio_en_especie_ann_input = widgets.FloatText(
    value=0,
    description='Retribució en espècie (€):',
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)

grup_cotitzacio_input = widgets.Dropdown(
    options=[
        "Sou mensual; adult; Enginyeres i llicenciades universitàries",
        "Sou mensual; adult; Enginyers tècnics, perits i ajudants titulats",
        "Sou mensual; adult; Caps administratius i de taller",
        "Sou mensual; adult; altres",
        "Base diaria; adult",
        "Menor d'edat"
    ],
    value="Sou mensual; adult; Enginyeres i llicenciades universitàries",
    description="Grup cotització:",
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)


contract_type_input = widgets.Dropdown(
    options=[("Indefinit", "indefinite"), ("Temporal", "temporary")],
    value="indefinite",
    description="Tipus contracte:",
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)

display(Markdown("Qualsevol deducció addicional. p.e plans de pensions..."))

other_annual_deductions_input = widgets.FloatText(
    value=0,
    description="Altres deduccions (€):",
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)

display(widgets.VBox([
    annual_gross_input,
    n_pagues_input,
    pagues_prorratejades_input,
    retribucio_en_especie_ann_input,
    grup_cotitzacio_input,
    contract_type_input,
    other_annual_deductions_input
]))

# -------------------------
# Situació familiar
# -------------------------
display(Markdown("### 👪 Situació familiar"))

age_input = widgets.IntSlider(
    value=30, min=16, max=100, step=1,
    description='Edat:',
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)

ascendents_input = widgets.IntSlider(
    value=0, min=0, max=5, step=1,
    description='Ascendents dependents:',
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)

disability_self_input = widgets.IntSlider(
    value=0, min=0, max=100, step=1,
    description='% Discapacitat pròpia:',
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)

disability_relatives_input = widgets.IntSlider(
    value=0, min=0, max=100, step=1,
    description='% Discapacitat familiars:',
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)

display(widgets.VBox([
    age_input,
    ascendents_input,
    disability_self_input,
    disability_relatives_input
]))

# -------------------------
# Fills
# -------------------------
display(Markdown("### 🧒 Fills"))
display(Markdown("Introdueix l'edat dels fills seperat per comes. p.e (12,22,26)"))
children_ages_input = widgets.Text(
    value="",
    description='Edat fills:',
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)
display(children_ages_input)
display(Markdown("Introdueix el percentatge de discapacitat dels fills (en cas de tenir-ne) també separada per comes. p.e (33,0,90)"))

children_disabilities_input = widgets.Text(
    value="",
    description='% discapacitat fills:',
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)


display(children_disabilities_input)

# -------------------------
# Botó i output
# -------------------------
button = widgets.Button(
    description="Calcular sou net",
    button_style='success',
    layout=widgets.Layout(width='200px')
)
output = widgets.Output()
display(button, output)


# -------------------------
# Funció que s'executa al pitjar el botó
# -------------------------
def on_button_click(b):
    with output:
        output.clear_output()
        # Conversió inputs
        gross = Decimal(str(annual_gross_input.value))
        n_pagues = int(n_pagues_input.value)
        pagues_prorratejades = bool(pagues_prorratejades_input.value)
        retribucio_en_especie_ann = Decimal(str(retribucio_en_especie_ann_input.value))
        age = int(age_input.value)
        ascendents = int(ascendents_input.value)
        disability_self = int(disability_self_input.value)
        disability_relatives = int(disability_relatives_input.value)
        grup_cotitzacio = grup_cotitzacio_input.value
        contract_type = contract_type_input.value
        other_deductions = Decimal(str(other_annual_deductions_input.value))

        # Fills
        try:
            children_ages = [int(x.strip()) for x in children_ages_input.value.split(",") if x.strip() != ""]
        except:
            children_ages = []
        try:
            children_disabilities = [int(x.strip()) for x in children_disabilities_input.value.split(",") if x.strip() != ""]
        except:
            children_disabilities = [0]*len(children_ages)

        # Crear objecte FamilySituation
        fam = FamilySituation(
            age=age,
            children_ages=children_ages,
            children_disabilities=children_disabilities,
            ascendents_dependent=ascendents,
            disability_percent_self=disability_self,
            disability_percent_relatives=disability_relatives
        )

        # Crida al compute_net_pay per obtenir el resultat
        result = compute_net_pay(
            basic_annual_gross=gross,
            n_pagues=n_pagues,
            pagues_prorratejades=pagues_prorratejades,
            retribucio_en_especie_ann=retribucio_en_especie_ann,
            grup_cotització=grup_cotitzacio,
            contract_type=contract_type,
            family=fam,
            region=ca_selector.value,
            other_annual_deductions=other_deductions
        )

button.on_click(on_button_click)

## 🔹 Calculadora de sou net 2025

Omple les dades a continuació. Prem el botó 'Calcular sou net' quan hagis acabat.

### 🌍 Comunitat autònoma

Selecciona la teva comunitat per aplicar l'IRPF correcte (estat + autonòmic).

ALERTA! De moment només funciona Catalunya.

...

Text(value='', description='Edat fills:', layout=Layout(width='600px'), style=DescriptionStyle(description_wid…

Introdueix el percentatge de discapacitat dels fills (en cas de tenir-ne) també separada per comes. p.e (33,0,90)

Text(value='', description='% discapacitat fills:', layout=Layout(width='600px'), style=DescriptionStyle(descr…

Button(button_style='success', description='Calcular sou net', layout=Layout(width='200px'), style=ButtonStyle…

Output()

In [36]:
#@title Càlcul d'increment de sou
import ipywidgets as widgets
from IPython.display import display, Markdown
from decimal import Decimal

display(Markdown("## 📈 Comparativa d'increment de sou"))
display(Markdown("Introdueix el teu sou brut anual anterior i el nou sou brut anual per calcular l'increment net."))

# Layout per als widgets d'increment
desc_width_inc = '250px'
widget_width_inc = '600px'

previous_gross_input = widgets.FloatText(
    value=25000,
    description='Sou brut anual anterior (€):',
    layout=widgets.Layout(width=widget_width_inc),
    style={'description_width': desc_width_inc}
)

new_gross_input = widgets.FloatText(
    value=28000,
    description='Nou sou brut anual (€):',
    layout=widgets.Layout(width=widget_width_inc),
    style={'description_width': desc_width_inc}
)

calculate_increment_button = widgets.Button(
    description="Calcular Increment Net",
    button_style='info',
    layout=widgets.Layout(width='250px')
)

increment_output = widgets.Output()

display(widgets.VBox([previous_gross_input, new_gross_input, calculate_increment_button]))
display(increment_output)

def on_calculate_increment_click(b):
    with increment_output:
        increment_output.clear_output()

        previous_gross = Decimal(str(previous_gross_input.value))
        new_gross = Decimal(str(new_gross_input.value))

        # Use the same parameters as the main calculator for consistency
        # You might want to add inputs for these if they can change
        n_pagues = int(n_pagues_input.value) # Assuming n_pagues_input is available from the cell above
        pagues_prorratejades = bool(pagues_prorratejades_input.value) # Assuming pagues_prorratejades_input is available
        retribucio_en_especie_ann = Decimal(str(retribucio_en_especie_ann_input.value)) # Assuming retribucio_en_especie_ann_input is available
        age = int(age_input.value) # Assuming age_input is available
        ascendents = int(ascendents_input.value) # Assuming ascendents_input is available
        disability_self = int(disability_self_input.value) # Assuming disability_self_input is available
        disability_relatives = int(disability_relatives_input.value) # Assuming disability_relatives_input is available
        grup_cotitzacio = grup_cotitzacio_input.value # Assuming grup_cotitzacio_input is available
        contract_type = contract_type_input.value # Assuming contract_type_input is available
        other_deductions = Decimal(str(other_annual_deductions_input.value)) # Assuming other_annual_deductions_input is available
        region = ca_selector.value # Assuming ca_selector is available

        fam = FamilySituation(
            age=age,
            children_ages=[int(x.strip()) for x in children_ages_input.value.split(",") if x.strip() != ""],
            children_disabilities=[int(x.strip()) for x in children_disabilities_input.value.split(",") if x.strip() != ""] if children_disabilities_input.value.strip() != "" else [0]*len([int(x.strip()) for x in children_ages_input.value.split(",") if x.strip() != ""]),
            ascendents_dependent=ascendents,
            disability_percent_self=disability_self,
            disability_percent_relatives=disability_relatives
        )


        previous_result = compute_net_pay(
            basic_annual_gross=previous_gross,
            n_pagues=n_pagues,
            pagues_prorratejades=pagues_prorratejades,
            retribucio_en_especie_ann=retribucio_en_especie_ann,
            grup_cotització=grup_cotitzacio,
            contract_type=contract_type,
            family=fam,
            region=region,
            other_annual_deductions=other_deductions,
            show_output=False # Suppress output
        )

        new_result = compute_net_pay(
            basic_annual_gross=new_gross,
            n_pagues=n_pagues,
            pagues_prorratejades=pagues_prorratejades,
            retribucio_en_especie_ann=retribucio_en_especie_ann,
            grup_cotització=grup_cotitzacio,
            contract_type=contract_type,
            family=fam,
            region=region,
            other_annual_deductions=other_deductions,
            show_output=False # Suppress output
        )

        increment_annual_net = new_result["sou_net_per_paga"] * Decimal(n_pagues) - previous_result["sou_net_per_paga"] * Decimal(n_pagues)
        increment_monthly_net = new_result["sou_net_mensual_equivalent"] - previous_result["sou_net_mensual_equivalent"]


        display(Markdown("### Resultat de l'increment"))
        print(f"Increment anual net: {increment_annual_net:.2f} €")
        print(f"Increment mensual net equivalent: {increment_monthly_net:.2f} €")


calculate_increment_button.on_click(on_calculate_increment_click)

## 📈 Comparativa d'increment de sou

Introdueix el teu sou brut anual anterior i el nou sou brut anual per calcular l'increment net.

Output()